## 4.1 누락된 데이터 다루기

### 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [15]:
import pandas as pd
from io import StringIO

csv_data = \
"""A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0, 11.0, 12.0,
"""
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [16]:
# null값 확인
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

In [17]:
df.dropna(axis=0) # axis=0 은 행 삭제, axis=1은 열 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [18]:
# 모든 열이 NaN일 때만 행을 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [19]:
# NaN이 아닌 값이 4개보다 작은 행을 삭제
df.dropna(thresh=4) 

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [20]:
# 특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 4.1.3 누락된 값 대체 
- 보간(interpolation) 기법

In [21]:
# 평균으로 대체하는 보간 기법
# 각 특성 열의 평균으로 누락된 값 대체  

# simpleimputer 클래스는 한 특성의 통계 값을 사용하여 누락된 값을 채운다.

from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

위의 결과에서 1,2,4번째 열의 수치 총합 -> 1+5+2+6+4+8 = 24

각 열의 평균 24 / 4 = 6

-> 3번째 열의 누락값 3으로 대체

In [22]:
from sklearn.preprocessing import FunctionTransformer
ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

In [23]:
imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

# 누락된 값의 위치 인덱스가 왜 1. , 0. 혹은 0. , 1. 으로 표현되는지? -> 대략 알 것 같음

array([[ 1. ,  2. ,  3. ,  4. ,  0. ,  0. ],
       [ 5. ,  6. ,  7.5,  8. ,  1. ,  0. ],
       [10. , 11. , 12. ,  6. ,  0. ,  1. ]])

In [25]:
imr.indicator_.features_

array([2, 3], dtype=int64)

In [26]:
# 누락된 값의 위치를 나타내는 배열을 반환 -> 이 배열 값이 imr의 transform 메서드가 반환한 배열의 마지막 두 열에 해당함

imr.indicator_.fit_transform(df.values)

array([[False, False],
       [ True, False],
       [False,  True]])

In [27]:
# 누락된 값을 대체한 배열을 원본 배열의 값으로 복원

imr.inverse_transform(imputed_data)

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [28]:
# IterativeImputer 클래스는 누락된 값이 있는 특성과 다른 특성을 사용하여 누락된 값을 예측한다. 

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

In [29]:
# KNNImputer 클래스는 k-최근접 이웃 방식을 사용하여 누락된 값을 채운다.
# 샘플 개수가 n_neighbors보다 작으면 simpleimputer (strategy='mean') 과 결과가 같다.

from sklearn.impute import KNNImputer
kimr = KNNImputer()
kimr.fit_transform(df.values)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [30]:
# 판다스의 fillna 메서드로 누락된 값 채우기

df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [31]:
df.fillna(method='backfill') # 누락값을 다음 행의 값으로 채운다.

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [32]:
df.fillna(method='ffill')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [34]:
df.fillna(method='ffill', axis=1) # 누락값을 이전 열의 값으로 채운다.

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


## 4.2 범주형 데이터 다루기

범주형 데이터 
1. 순서가 있는 범주형 데이터 ex. 티셔츠의 사이즈 (XL > L)
2. 순서가 없는 범주형 데이터 ex. 색깔

In [45]:
import pandas as pd
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']])
df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### 4.2.2 순서가 있는 특성 매핑

In [46]:
# size feature의 산술적인 관계를 XL = L + 1 = M + 2 라고 가정

size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [47]:
size_mapping.items()

dict_items([('XL', 3), ('L', 2), ('M', 1)])

In [49]:
# 원래의 레이블로 복원

inv_size_mapping = {v: k  for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object